# Batch sizing enabled with multiple products and setup times

### Objetivos:

- **Modelar** soluções para o problema de programação linear para _BS_.
- **Implementar** a solução do problema em Julia.

O desafio do planejamento de produção de múltiplos itens em empresas que fabricam uma variedade de produtos é otimizar os recursos para maximizar os lucros ou minimizar os custos. Isso requer que os alunos formalizem, formulem e implementem uma solução em código. O problema envolve produtos com custos fixos, custos variáveis, custos de estoque, tempos de setup e tempos de produção, além de restrições de tempo de produção por período.

Com isso deseja-se determinar o planejamento de produção de múltiplos itens em um horizonte de planejamento de _T_ períodos:

- A produção de cada produto em um determinado período incorre um custo fixo que independe da quantidade e um custo por quantidade produzida;
- Há custos de estoque para cada produto por quantidade em inventário;
- Há um tempo gasto para o setup de cada produto bem como um tempo por unidade produzida do produto;
- Há um limite no tempo total utilizado na produção em cada período.

O programa solução calculará a quantidade a ser produzida de cada produto em cada período, com o objetivo de minimizar os custos totais, respeitando todas as restrições. Isso requer a aplicação de conhecimentos em otimização e programação linear.

Fique a vontade para checar os [rascunhos](https://www.canva.com/design/DAFvpFclQdA/YVYGJANahDe6_JfbGyDtPA/edit?utm_content=DAFvpFclQdA&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton) escritos durante a modelagem do problema original.

## Variáveis do problema:

- Descreve a quantidade **produzida** de um item _j_ no dia _i_:
$$qntprod_{item_jdia_i} \in \mathbb{N}$$

- Descreve a quantidade **estocada** de um item _j_ no dia _i_:
$$qntstocada_{item _jdia_i}\in \mathbb{N}$$

- Indica se ouve ou não **produção** de um item _j_ no dia _i_:
$$houveprod_{item _jdia_i} \in \{0,1\}$$

## Função Objetivo:

O objetivo do problema é reduzir o custo total da operação com base na caracteristicas do problema. Desta forma, definimos a função objetivo:

\begin{align*}
custototal(args[]) = \min \{\sum_{j=1}^{itens} \sum_{i=1}^{períodos} custoprod_{item _jdia_i}.qntprod_{item_jdia_i} + qntstocada_{item _jdia_i}.custostoque_{item_jdia_i} + custosetup_{item _jdia_i}*houveprod_{item _jdia_i}\}
\end{align*}

A fórmula leva em consideração o custo de produção para cada item produzido, a custo de armazenamento de cada produto e o custo de setup se houve produção no dia para cada combinação de _item_ ( $j$ ) e _período_ ( $i$ ), para que cada item diferente em cada período seja considerada em suas especificidades. Por isso, os argumentos _args_ da função são descritos por:

\begin{align*}
args[] = [demanda[itens][periodos], custosetup[itens][periodos], custoprod[itens][periodos], custostoque[itens][periodos], tempoprod[itens], temposetup[itens], tempomax[periodos]]  
\end{align*}

## Restrições:

Para garantir que as condições sejam satisfeitas, foram modeladas <strong>restrições</strong>. Para assegurar que o tempo de produção não seja superior ao tempo máximo daquele período, estabeleceu-se a seguinte restrição:

\begin{align*}
tempomax_{dia_i} \geq \sum_{j=1}^{itens} tempoprod_{item_j}.qntprod_{item_jdia_i} + \sum_{j=1}^{itens} custosetup_{dia_i}.houveprod_{item _jdia_i}
\end{align*}

Para garantir que a demanda de cada dia seja cumprida, bem como para manter a relação entre o armazenamento de um período para outro e a demanda do dia, foi modelada as seguintes restrições:

\begin{align*}
qntstocada_{item _jdia_1} = qntprod_{item_jdia_1} - demanda_{item_jdia_1}
\end{align*}

\begin{align*}
qntstocada_{item _jdia_i} = qntprod_{item_jdia_i} - demanda_{item_jdia_i} + qntstocada_{item _jdia_i-1}  \forall i > 1
\end{align*}

Por último, para garantir que o solver tenha que calcular os custos de configuração do item nos dias em que haja produção, se houver produção no dia, a variável _houveprod_ deve ser igual a 1. Para isso, utilizamos a seguinte restrição:

\begin{align*}
qntprod_{item_jdia_i} \geq \sum_{i=1}^{períodos} demanda_{item_jdia_i}.houveprod_{item _jdia_i}
\end{align*}


## Implementação em Julia:

Prepare o ambiente de execução (Google Colab):

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.7" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Julia `julia -v` instalado com sucesso!"
  echo "vá para a seção 'Verificação da Instalação'."
fi

Importe os pacotes do Solver de problemas lineares:

In [ ]:
import Pkg; Pkg.add("JuMP"); Pkg.add("Cbc")

Declare o modelo, e defina tempo limite para processamento:

In [ ]:
using JuMP, Cbc
model = Model(Cbc.Optimizer)
set_time_limit_sec(model,30)

Exemplos de entrada:

In [ ]:
# Exemplo 1:
n_items = 3
n_periods = 7
demand = [118 85 110 77 92 99 94; 123 0 97 0 78 122 120; 0 85 125 95 117 119 78]
setup_cost = [1000.0 800.0 500.0 700.0 100.0 500.0 800.0; 800.0 400.0 800.0 600.0 400.0 600.0 800.0; 200.0 400.0 300.0 600.0 1000.0 1000.0 300.0]
perunit_cost = [0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
storage_cost = [4.0 4.0 1.0 4.0 2.0 4.0 4.0; 2.0 5.0 2.0 3.0 2.0 4.0 4.0; 4.0 5.0 3.0 1.0 4.0 3.0 5.0]
setup_time = [40.0, 40.0, 10.0]
perunit_time = [1.0, 1.0, 1.0]
capacity = [469, 469, 469, 469, 469, 469, 469]

In [ ]:
# Exemplo 2:
n_items = 10
n_periods = 15
demand = [0 81 94 0 99 123 91 118 123 114 111 119 77 81 113; 86 0 0 97 77 121 115 115 101 86 98 125 122 94 114; 0 76 113 93 94 103 113 116 86 98 112 105 117 125 94; 86 123 112 97 80 116 111 98 94 116 91 80 107 79 108; 97 0 120 0 123 109 125 89 94 110 90 78 105 92 121; 83 86 0 88 92 123 116 120 101 89 81 111 77 88 79; 76 79 90 88 93 112 123 87 121 77 110 80 93 111 108; 119 92 0 92 117 104 84 123 93 78 98 107 104 107 90; 0 124 78 107 114 90 102 103 109 108 100 122 81 114 83; 97 100 0 85 114 104 109 92 95 87 117 111 94 116 107]
setup_cost = [700.0 700.0 400.0 400.0 200.0 100.0 900.0 100.0 100.0 400.0 300.0 500.0 700.0 300.0 300.0; 700.0 200.0 200.0 400.0 500.0 1000.0 100.0 100.0 400.0 200.0 800.0 700.0 500.0 800.0 100.0; 800.0 700.0 700.0 600.0 1000.0 400.0 500.0 600.0 800.0 100.0 500.0 600.0 300.0 1000.0 800.0; 400.0 100.0 400.0 900.0 500.0 500.0 200.0 900.0 100.0 300.0 800.0 700.0 900.0 100.0 500.0; 700.0 800.0 300.0 600.0 700.0 800.0 600.0 100.0 300.0 500.0 800.0 700.0 1000.0 400.0 1000.0; 400.0 600.0 900.0 600.0 100.0 100.0 100.0 100.0 900.0 800.0 600.0 200.0 500.0 1000.0 800.0; 200.0 600.0 200.0 500.0 700.0 300.0 800.0 100.0 300.0 700.0 600.0 800.0 1000.0 600.0 400.0; 700.0 400.0 700.0 1000.0 200.0 700.0 400.0 600.0 500.0 1000.0 400.0 700.0 800.0 800.0 300.0; 600.0 200.0 900.0 300.0 700.0 500.0 200.0 400.0 200.0 1000.0 800.0 700.0 100.0 700.0 800.0; 200.0 100.0 400.0 400.0 700.0 500.0 600.0 900.0 300.0 500.0 800.0 1000.0 300.0 800.0 1000.0]
perunit_cost = [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
storage_cost = [2.0 3.0 5.0 5.0 3.0 3.0 4.0 1.0 1.0 3.0 3.0 5.0 1.0 3.0 1.0; 5.0 2.0 2.0 2.0 1.0 1.0 1.0 2.0 2.0 1.0 4.0 5.0 3.0 4.0 2.0; 3.0 2.0 2.0 3.0 5.0 5.0 2.0 2.0 3.0 5.0 4.0 5.0 2.0 1.0 3.0; 4.0 1.0 4.0 5.0 5.0 5.0 4.0 5.0 3.0 1.0 4.0 3.0 5.0 1.0 2.0; 3.0 2.0 1.0 3.0 2.0 2.0 1.0 3.0 5.0 2.0 5.0 1.0 4.0 5.0 3.0; 1.0 5.0 3.0 4.0 2.0 4.0 1.0 1.0 2.0 4.0 2.0 4.0 4.0 5.0 3.0; 3.0 1.0 2.0 3.0 1.0 3.0 4.0 3.0 5.0 3.0 4.0 3.0 3.0 1.0 3.0; 5.0 5.0 3.0 5.0 2.0 2.0 5.0 3.0 5.0 4.0 5.0 2.0 4.0 2.0 5.0; 1.0 3.0 1.0 2.0 1.0 2.0 4.0 4.0 5.0 5.0 2.0 1.0 1.0 1.0 2.0; 3.0 4.0 2.0 5.0 4.0 2.0 2.0 3.0 1.0 4.0 5.0 4.0 3.0 4.0 4.0]
setup_time = [10.0, 40.0, 20.0, 50.0, 10.0, 20.0, 40.0, 50.0, 10.0, 40.0]
perunit_time = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
capacity = [1114, 1114, 1114, 1114, 1114, 1114, 1114, 1114, 1114, 1114, 1114, 1114, 1114, 1114, 1114]

Declare as variáveis:

In [ ]:
@variable(model,z)
@variable(model,x[i in 1:n_items, j in 1:n_periods],Bin)
@variable(model,y[i in 1:n_items, j in 1:n_periods] >= 0)
@variable(model,storage[i in 1:n_items, j in 1:n_periods] >=0 )

Declare a função objetivo:

In [ ]:
z = sum( sum(perunit_cost[i,j]y[i,j] + storage_cost[i,j]storage[i,j] + setup_cost[i,j]x[i,j] for i in 1:n_items) for j in 1:n_periods)
@objective(model,Min, z)

Declare as restrições do modelo:

In [ ]:
# No primeiro período, a produção não leva em conta o estoque do período anterior
@constraint(model, first_period_production[i in 1:n_items],
    y[i, 1] == demand[i, 1] + storage[i, 1])

# Monta a restrição para o i-ésimo produto no j-ésimo período
@constraint(model, storage_constraint[i in 1:n_items, j in 2:n_periods],
    storage[i, j-1] + y[i, j] == demand[i, j] + storage[i, j])

# Não deve sobrar nada no estoque ao fim dos períodos
@constraint(model, final_storage_empty[i in 1:n_items],
    storage[i, n_periods] == 0)

# Garante que x[i, j] seja 1 caso algo seja produzido naquele período
@constraint(model, production_indicator[i in 1:n_items, j in 1:n_periods],
    y[i, j] <= 800000 * x[i, j])

# Garante que não exceda o tempo limite
@constraint(model, time_limit_constraint[j in 1:n_periods],
    capacity[j] >= sum(setup_time[i] * x[i, j] + perunit_time[i] * y[i, j] for i in 1:n_items))


Imprima detalhes do modelo:

In [ ]:
print(model)

Otimize o modelo:

In [ ]:
JuMP.optimize!(model)

Verifique os resultados:

In [ ]:
for i in 1:n_items
  for j in 1:n_periods
    if value(y[i,j])> 0.01
      println("y",i," ",j," ",value(y[i,j]))
    end
  end
end

---

[Fernando Schettini](https://linktr.ee/fernandoschett) <br/>
Inter at the Supercomputing Center for Industrial Innovation SENAI-CIMATEC [CS2I](https://www.senaicimatec.com.br/).<br/>
[João Bernardino]() <br/>
Computer Science Student at UFBA.<br/>